In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [8]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp_2022/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['mar']
sheet = 'Sheet'

In [3]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [84]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[(df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index().groupby('snils').max().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[(sums.stv >= 1.0) & (sums.status == 'Работа') & (sums.day == 1) & (sums.status_pref == 'Работа')]
    sums = sums[['inn_x', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [5]:
def create_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data(months[i])
        else:
            res = pd.merge(res, load_data(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [6]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [7]:
def load_groups(file):
    df = pd.read_excel(path_to_docs + file + '.xlsx', sheet_name='УГД')
    ped = df[df['Педагогический'] == 1]['Должность'].tolist()
    isp = df[df['Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми'] == 1]['Должность'].tolist()
    aup = df[df['УГД'] == 'Административно-управленческий персонал']['Должность'].to_list()
    return ped, isp, aup

In [13]:
def szp(row, months):
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    cnt = 0
    sum_szp = 0
    for month in months:
        if pd.isna(row['sum_' + month]):
            continue
        cnt += 1
        sum_szp += row['sum_' + month]
    # if row['inn'] == 9715217689 and row['snils'] == '173-260-871 66':
    #     print(cnt, sum_szp)
    return sum_szp / cnt

In [14]:
def szp_ped(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ped:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum_szp / cnt

In [15]:
def szp_teach(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] != 'Учитель':
        return np.nan
    return sum_szp / cnt

In [24]:
def szp_class(row, months):
    if row['snils'] not in is_class_ruk:
        return np.nan
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] != 'Учитель':
        return np.nan
    return sum_szp / cnt

In [16]:
ped, isp, aup = load_groups('input')

In [9]:
is_class_ruk = pd.read_excel(path_to_data + 'mar_class.xlsx')['snils'].to_list()
is_class_ruk

['001-173-705 86',
 '001-852-207 00',
 '001-906-648 18',
 '002-028-822 86',
 '002-028-997 07',
 '002-029-013 65',
 '002-029-015 67',
 '002-843-223 06',
 '004-479-803 49',
 '004-479-807 53',
 '004-479-810 48',
 '004-512-180 90',
 '005-739-494 61',
 '008-082-680 37',
 '012-520-728 95',
 '015-727-104 29',
 '016-340-841 20',
 '018-531-139 30',
 '018-533-607 45',
 '018-533-622 44',
 '018-533-664 54',
 '018-533-668 58',
 '018-533-669 59',
 '018-533-750 51',
 '019-002-754 13',
 '019-106-837 37',
 '019-441-818 52',
 '019-462-813 61',
 '020-672-548 25',
 '020-987-041 46',
 '022-380-753 21',
 '022-589-634 63',
 '023-526-544 27',
 '023-978-958 01',
 '024-605-396 32',
 '024-932-633 47',
 '024-932-637 51',
 '024-932-662 52',
 '024-932-675 57',
 '024-932-679 61',
 '024-932-685 59',
 '024-932-687 61',
 '024-975-338 75',
 '025-068-212 22',
 '025-625-568 51',
 '028-440-604 37',
 '028-448-462 73',
 '029-268-579 90',
 '030-709-453 26',
 '032-614-686 34',
 '034-414-169 20',
 '040-541-694 25',
 '052-443-56

In [10]:
len(is_class_ruk)

38299

In [85]:
res = create_res(months)

In [86]:
res['szp'] = res.apply(lambda row: szp(row, months), axis= 1)
res['szp_ped'] = res.apply(lambda row: szp_ped(row, months), axis=1)
res['szp_teach'] = res.apply(lambda row: szp_teach(row, months), axis=1)

In [87]:
res['szp_class'] = res.apply(lambda row: szp_class(row, months), axis=1)

In [26]:
res

,inn,snils,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach,szp_class
1,7718309983,001-005-786 70,85809.12,1.0,Учитель,85809.12,85809.12,85809.12,NaN
3,7736614860,001-010-312 25,160599.30,1.0,Учитель,160599.30,160599.30,160599.30,160599.30
4,7723169690,001-011-996 67,166752.15,1.0,Учитель,166752.15,166752.15,166752.15,166752.15
5,7727123537,001-017-453 65,86627.92,1.0,Секретарь руководителя,86627.92,NaN,NaN,NaN
6,7733088746,001-017-484 72,86787.10,1.0,Учитель,86787.10,86787.10,86787.10,NaN
...,...,...,...,...,...,...,...,...,...
165377,7743024406,214-822-829 56,54500.00,1.0,Уборщик служебных и производственных помещений,54500.00,NaN,NaN,NaN
165378,7714890087,214-860-989 83,71680.00,1.0,Рабочий по комплексному обслуживанию и ремонту...,71680.00,NaN,NaN,NaN
165380,7709944805,214-887-443 92,54536.00,1.0,Уборщик служебных и производственных помещений,54536.00,NaN,NaN,NaN
165381,7734247614,214-988-963 20,65000.00,1.0,Уборщик территории,65000.00,NaN,NaN,NaN


In [88]:
szp_class_ruk = res[res.szp_class.isna() == False][['inn', 'szp_class']].groupby('inn').mean()

In [89]:
tq01 = res[res.szp_teach.isna() == False][['inn', 'szp_teach']].groupby('inn').quantile(0.1).rename(columns={'szp_teach': 'tq01'})
szp_10_teach = pd.merge(res[res.szp_teach.isna() == False][['inn', 'szp_teach']], tq01, how='left', on='inn')
szp_10_teach = szp_10_teach[szp_10_teach.szp_teach <= szp_10_teach.tq01][['inn', 'szp_teach']].groupby('inn').mean()

In [90]:
cq01 = res[res.szp_teach.isna() == False][['inn', 'szp_class']].groupby('inn').quantile(0.1).rename(columns={'szp_class': 'cq01'})
szp_10_class = pd.merge(res[res.szp_class.isna() == False][['inn', 'szp_class']], cq01, how='left', on='inn')
szp_10_class = szp_10_class[szp_10_class.szp_class <= szp_10_class.cq01][['inn', 'szp_class']].groupby('inn').mean()

In [91]:
szp_teach_min = res[(res.szp_teach.isna() == False) & (res.szp_teach)][['inn', 'szp_teach']].groupby('inn').min()

In [92]:
szp_class_ruk_min = res[(res.szp_class.isna() == False) & (res.szp_class)][['inn', 'szp_class']].groupby('inn').min()

In [95]:
szp_200_teach = res[(res.szp_teach.isna() == False) & (res.szp_teach > 200000)][['inn', 'szp_teach']].groupby('inn').count()

In [98]:
szp_200_nped = res[(res.job_mar.isin(ped) == False) & (res.szp > 200000)][['inn', 'szp']].groupby('inn').count()

In [100]:
output = 'C:/Users/PetukhovMD/Desktop/szp_2022/1 квартал/'
print_df(szp_200_teach, output + 'szp_200_teach')
print_df(szp_200_nped, output + 'szp_200_nped')
print_df(szp_10_class, output + 'szp_10_class')
print_df(szp_10_teach, output + 'szp_10_teach')
print_df(szp_class_ruk_min, output + 'szp_class_ruk_min')
print_df(szp_teach_min, output + 'szp_teach_min')
print_df(szp_class_ruk, output + 'szp_class_ruk')

In [ ]:
teach_szp = pd.DataFrame()

In [97]:
szp_200_teach.describe()

,szp_teach
count,207.000000
mean,3.536232
std,3.522107
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,31.000000


In [94]:
szp_teach_min.describe()

,szp_teach
count,517.000000
mean,85544.841103
std,6213.985349
min,54389.580000
25%,84104.800000
50%,84199.950000
75%,86605.740000
max,127804.000000


In [93]:
szp_class_ruk_min.describe()

,szp_class
count,515.000000
mean,97273.400524
std,10818.189519
min,73423.350000
25%,89298.695000
50%,95943.880000
75%,103842.220000
max,163308.670000


In [42]:
eko_sums = pd.read_excel(path_to_data + 'eko_sums.xlsx')

In [43]:
snils_in_res = res['snils'].to_list()

In [44]:
eko_sums[eko_sums.snils.isin(snils_in_res)].describe()

,sums
count,570.000000
mean,80057.195000
std,23010.843972
min,17072.730000
25%,65500.000000
50%,77596.000000
75%,89115.000000
max,235075.400000
